In [1]:
from typing import List, Dict
from pixels import *
from fmatrix import *
import random
import numpy as np
import json

In [2]:
def find_matrix_center(matrix:Matrix):
    if matrix.size % 2 != 0:
        return (matrix.size//2,)*2
    else:
        raise Exception("Non odd sized matrix")

In [3]:
class Board:
    def __init__(self, tile_options:List[Dict]):
        # TODO: Change fixed size of 5x5
        self.__m__ = np.tile(-1, (5,5)).tolist()
        self.__m__ = Matrix(self.__m__)
        self.__options__ = tile_options
    
    def global_decision_space(self) -> List[int]:
        return list(range(len(self.__options__)))
    
    def cell_decision_space(self, pos:Tuple[int,int]) -> Set[int]:
        return set()

    def collapse(self, pos:Tuple[int,int], decision_space=List[int]):
        if pos[0] > self.__m__.size or pos[1] > self.__m__.size:
            raise Exception("Invalid position")

        cell = self.__m__[pos]

        if cell != -1: # Collapsed cell
            return self
        else: # Uncollapsed cell
            choice = random.choice(decision_space)
            self.__m__.setAt(pos = pos,
                    n   = choice)
            return self

    def collapse_center(self, initial_decisions:List[int]):
        return self.collapse(pos = find_matrix_center(self.__m__), decision_space = initial_decisions)
    
    def collapse_neighbours(self, pos:Tuple[int,int]):
        cell = self.__m__[pos]
        if cell == -1:
            raise Exception("Cannot collapse neighbours of non-collapsed cell.")
        
        # TODO: fmatrix reverse coordinates
        y, x = pos

        up_decision    = self.__options__[cell]['connections'][0]
        right_decision = self.__options__[cell]['connections'][1]
        down_decision  = self.__options__[cell]['connections'][2]
        left_decision  = self.__options__[cell]['connections'][3]

        self.collapse(pos=(y-1, x+0), decision_space = up_decision)
        self.collapse(pos=(y+0, x+1), decision_space = right_decision)
        self.collapse(pos=(y+1, x+0), decision_space = down_decision)
        self.collapse(pos=(y+0, x-1), decision_space = left_decision)
        return self
    

In [4]:
tiles = []
with open('tile-connections.json', 'r') as fp:
    tiles = json.load(fp=fp)

In [5]:
b = Board(tiles)
b.__m__.rows()

[[-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1]]

In [6]:
b.collapse_center(list(range(len(tiles))))
b.__m__.rows()

[[-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, 18, -1, -1],
 [-1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1]]

In [7]:
b.collapse_neighbours((2,2))
b.__m__.rows()

[[-1, -1, -1, -1, -1],
 [-1, -1, 23, -1, -1],
 [-1, 27, 18, 64, -1],
 [-1, -1, 9, -1, -1],
 [-1, -1, -1, -1, -1]]

In [8]:
# TODO: tile_42 left connections includes invalid tile 34
# TODO: Take the matrix:
# [[-1, -1, -1, -1, -1],
#  [-1, -1, 23, -1, -1],
#  [-1, 27, 18, 64, -1],
#  [-1, -1, 9, -1, -1],
#  [-1, -1, -1, -1, -1]]
# The decision space of [3,3] is the intersection of the decision spaces of [2,3] and [3,2]
# The decision space of a cell which value is -1 should be the 